pmgus.ipynb
"Pre-Market Gap-Up Screener"

In [1]:
import pandas as pd
import numpy as np

In [2]:
# read data file from TradingView to pandas df
""" 
Market : US
Exchange : NASDAQ, NYSE
Pre-market Chg > $0.00 USD

Stocks = ~1100
"""
base_file_path = '/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/'
file_name =  'tv_screen_gap-up_2024-10-29.csv'
pmgus_df = pd.read_csv(base_file_path + file_name)


print(len(pmgus_df))
pmgus_df.tail(2)

1131


,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,...,"Bollinger Bands (20) 5 minutes, Basis","Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes
1129,ZENA,"ZenaTech, Inc.",Information technology services,Technology services,NASDAQ,"NASDAQ Composite, NASDAQ Computer",NaN,NaN,2.11,USD,...,2.192510,2.108617,CA98936T2083,0.156557,NaN,NaN,NaN,NaN,NaN,Sell
1130,ECC/PD,Eagle Point Credit Company Inc. 6.75% Series D...,Investment trusts/Mutual funds,Miscellaneous,NYSE,NaN,NaN,NaN,19.94,USD,...,19.923175,19.900088,US2698098025,0.473476,NaN,NaN,NaN,NaN,NaN,Sell


In [4]:
def categorize_market_cap(df):
    """Categorize stocks based on market capitalization."""
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    # drop Undefined
    df = df[df['marketCapType'] != 'Undefined']
    # convert to numeric the Market capitalization column
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'])
    
    return df

In [5]:
# categorize market cap
pmgus_df = categorize_market_cap(pmgus_df).copy()


print(len(pmgus_df))
pmgus_df.head()

1120


/var/folders/8w/4mj_4m4d0rs1q22bl9g77hhw0000gn/T/ipykernel_40607/3074008106.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Market capitalization'] = pd.to_numeric(df['Market capitalization'])


,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,...,"Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes,marketCapType
0,GOOGL,Alphabet Inc.,Internet software/Services,Technology services,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500...",2.061371e+12,USD,166.72,USD,...,166.704687,US02079K3059,1.648845,2024-07-23,2024-10-29,Buy,202.105263,USD,Sell,Titans
1,GOOG,Alphabet Inc.,Internet software/Services,Technology services,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500...",2.061371e+12,USD,168.34,USD,...,168.351993,US02079K1079,1.649972,2024-07-23,2024-10-29,Buy,201.928571,USD,Sell,Titans
2,META,"Meta Platforms, Inc.",Internet software/Services,Technology services,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500...",1.462639e+12,USD,578.16,USD,...,578.243656,US30303M1027,1.150212,2024-07-31,2024-10-30,Strong buy,613.132712,USD,Sell,Titans
3,LLY,Eli Lilly and Company,Pharmaceuticals: major,Health technology,NYSE,"S&P 500, S&P 500 Health Care, S&P 100, Nasdaq ...",8.511918e+11,USD,895.59,USD,...,894.883320,US5324571083,1.062491,2024-08-08,2024-10-30,Buy,1020.547600,USD,Sell,Titans
4,TSLA,"Tesla, Inc.",Motor vehicles,Consumer durables,NASDAQ,"S&P 500, NASDAQ 100, NASDAQ Composite, S&P 500...",8.426728e+11,USD,262.51,USD,...,262.101556,US88160R1014,1.286275,2024-10-23,2025-01-22,Neutral,227.427273,USD,Sell,Titans


you adapted the commented code already start with refinement of the conditions

also use the yahoo col names not your own program ofp bull shit. 

In [7]:
# check for undefined market cap
pmgus_df[pmgus_df['marketCapType'] == 'Undefined']

,Symbol,Description,Industry,Sector,Exchange,Index,Market capitalization,Market capitalization - Currency,Price,Price - Currency,...,"Bollinger Bands (20) 5 minutes, Lower",International Securities Identification Number,Relative Volume 1 day.1,Recent earnings date,Upcoming earnings date,Analyst Rating,Target price 1 year,Target price 1 year - Currency,Technical Rating 5 minutes,marketCapType


In [ ]:
# data - converting some cols to numeric + volume consolidation


In [5]:
# execute categorization
category_setup_df = categorize_market_cap(pmgus_df).copy()
# # drop Undefined marketCapType
# category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume'
]

# apply conversion
category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

In [7]:
# Criteria configuration for each market cap category
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    # "Midlers" in TradingView
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

def filter_stocks(df, config):
    """Filter stocks based on configuration criteria."""
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

In [8]:
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# execute filtering
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# add 'Market capitalization' to the reordered columns list
cols_list = [
    'Symbol', 'Description', 'Pre-market Change %', 'Pre-market Gap %', 'marketCapType',
    'Market capitalization','Price', 'Pre-market Open', 'Industry', 'Index', 'Sector', 
    'Exchange','Recent earnings date', 'Upcoming earnings date', 'Float shares outstanding', 
    'Average Volume 10 days', 'Average Volume 30 days', 'Average Volume 90 days',
    'Relative Volume 1 day', 'Relative Volume 5 minutes', 'Relative Volume 30 minutes', 
    'Relative Volume at Time'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Market capitalization', 'Pre-market Change %'],
    ascending=[False, False]).reset_index(drop=True)

print(f"{smash_df.shape[0]} stockes found in the screener.")


## ISSUE ##
#### HARD CODED FILE PATH ####
# also change this if you add another level, you probably will to screener_smoke 
# output_filename = f'pre_market_gap_up_screener_targets_{today_date_str}.csv'
output_filename = f'/Users/sudz4/Desktop/SPS_local/sps/x_pre_market_gap_up_screener/tv_screen_gap-up_2024-10-28_NEW.csv'

smash_df.to_csv(output_filename, index=False)

53 stockes found in the screener.


In [10]:
smash_df.columns

Index(['Symbol', 'Description', 'Pre-market Change %', 'Pre-market Gap %',
       'marketCapType', 'Market capitalization', 'Price', 'Pre-market Open',
       'Industry', 'Index', 'Sector', 'Exchange', 'Recent earnings date',
       'Upcoming earnings date', 'Float shares outstanding',
       'Average Volume 10 days', 'Average Volume 30 days',
       'Average Volume 90 days', 'Relative Volume 1 day',
       'Relative Volume 5 minutes', 'Relative Volume 30 minutes',
       'Relative Volume at Time'],
      dtype='object')

In [11]:
smash_df.head(2)

,Symbol,Description,Pre-market Change %,Pre-market Gap %,marketCapType,Market capitalization,Price,Pre-market Open,Industry,Index,...,Recent earnings date,Upcoming earnings date,Float shares outstanding,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Relative Volume 1 day,Relative Volume 5 minutes,Relative Volume 30 minutes,Relative Volume at Time
0,PAYC,"Paycom Software, Inc.",0.005976,0.005976,Mid caps,9.611319e+09,167.35,167.36,Packaged software,"S&P 500, S&P 500 Industrials, ISE CTA Cloud Co...",...,2024-07-31,2024-10-30,5.054545e+07,501351.8,5.584963e+05,7.466308e+05,1.048406,8.938356,5.612154,1.344612
1,DJT,Trump Media & Technology Group Corp.,12.521115,6.862331,Mid caps,9.479493e+09,47.36,50.61,Advertising/Marketing services,"NASDAQ Composite, Nasdaq US Small Cap Growth, ...",...,NaN,NaN,7.514479e+07,57340062.0,3.509499e+07,1.764823e+07,2.112787,2.956046,1.192484,2.047502
